In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

import pickle
#import ml_metrics

In [2]:
df_results = pd.read_csv('result_images.csv')
df_results.head()

,Image number,Style label,Image Url,13264663777,13263750995,13263613156,13249068738,13244840492,13244520859,13241149237,...,13237797264,13237763648,13237765133,13237751174,13237465898,13237529736,13237464563,13237445953,13237375572,13237356060
0,1,"Comfortable,Casual,Loose-fit",https://images.afound.com/images/5b6ee9d2-51c4...,True,True,True,False,False,True,False,...,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated
1,2,"Feminine,Sexy,Party",https://na-kd.com/globalassets/201102_angelica...,False,False,False,False,True,False,False,...,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated
2,3,"Loose-fit,Playful,Comfortable",https://na-kd.com/globalassets/nakd_animal_pri...,False,True,False,False,True,True,False,...,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated
3,4,"Business,Chic,Colorful",https://cdn.gutsgusto.com/pictures/medium/8134...,False,False,False,False,True,True,False,...,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated
4,5,"Bold,Monochrome,Form-fitting",https://cdn.gutsgusto.com/pictures/medium/8206...,False,False,True,False,True,False,False,...,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated,Not rated


In [3]:
#create column style by splitting style label
df = df_results.assign(style=df_results['Style label'].str.split(',')).explode('style')

In [4]:
#remove first 3 columns
df = df.iloc[:, 3:]

In [5]:
#replace True for 1, False for -1 and Not rated for 0.
df = df.replace({'True':1,'False':-1,'Not rated':0})


In [6]:
df.head()

,13264663777,13263750995,13263613156,13249068738,13244840492,13244520859,13241149237,13240718620,13240463751,13240304890,...,13237763648,13237765133,13237751174,13237465898,13237529736,13237464563,13237445953,13237375572,13237356060,style
0,1,1,1,-1,-1,1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,Comfortable
0,1,1,1,-1,-1,1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,Casual
0,1,1,1,-1,-1,1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,Loose-fit
1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,Feminine
1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,Sexy


In [7]:
#get mean of all styles grouped together
df_mean = df.groupby('style').mean().reset_index()

In [8]:
style_indexes = df_mean['style'].to_dict()

In [9]:
style_indexes

{0: 'Active',
 1: 'Bohemian',
 2: 'Bold',
 3: 'Business',
 4: 'Casual',
 5: 'Chic',
 6: 'Classic',
 7: 'Colorful',
 8: 'Comfortable',
 9: 'Contemporary',
 10: 'Designer',
 11: 'Eclectic',
 12: 'Feminine',
 13: 'Form-fitting',
 14: 'Formal',
 15: 'Goth',
 16: 'Loose-fit',
 17: 'Luxury',
 18: 'Minimalistic',
 19: 'Modest',
 20: 'Monochrome',
 21: 'Party',
 22: 'Playful',
 23: 'Romantic',
 24: 'Sexy'}

In [10]:
#save style indexes to pickle file
pickle.dump(style_indexes, open("style_indexes.p","wb"))

In [11]:
#transpose dataframe so it can be used on knn model
df_train = df_mean.drop(columns='style').T

In [12]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
13264663777,-0.666667,0.0,-0.142857,0.142857,0.035714,0.043478,-0.142857,-0.2,0.024390,0.0,...,0.0,0.035714,0.0,0.25,0.0,-1.0,-0.2,-0.181818,0.428571,0.111111
13263750995,-0.666667,0.0,-0.142857,-0.428571,-0.035714,-0.043478,-0.142857,-0.1,-0.024390,0.0,...,0.0,0.107143,0.0,0.00,0.0,-1.0,-0.2,0.000000,0.142857,0.111111
13263613156,0.000000,0.0,0.142857,-0.714286,0.107143,-0.130435,-0.142857,0.1,0.121951,0.0,...,0.0,0.250000,0.0,0.25,0.0,1.0,-0.2,0.000000,-0.428571,-0.333333
13249068738,-0.666667,0.0,-0.142857,-0.714286,-0.214286,-0.304348,-0.142857,-0.2,-0.268293,0.0,...,0.0,-0.321429,0.0,-0.25,0.0,-1.0,-0.2,-0.181818,-0.428571,-0.333333
13244840492,0.666667,0.0,0.142857,0.142857,0.000000,0.130435,-0.142857,0.0,0.073171,0.0,...,0.0,0.178571,0.0,-0.25,0.0,1.0,0.2,0.000000,0.142857,0.333333


In [13]:
#schrijf de matrix naar csv file
df_train.to_csv('style_like_scores_matrix.csv',index=True)

In [14]:
#split data in train and test set
train, test = train_test_split(df_train, test_size=0.3, random_state=42)


In [15]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
13244520859,0.666667,0.0,-0.142857,0.142857,0.071429,0.043478,-0.142857,0.1,0.121951,0.0,...,0.0,0.178571,0.0,0.000,0.0,-1.0,-0.2,0.000000,-0.142857,0.111111
13237319909,0.000000,0.0,0.000000,0.142857,0.089286,0.043478,0.142857,-0.1,0.097561,0.5,...,0.0,-0.035714,0.0,0.125,1.0,0.0,0.4,-0.090909,-0.285714,0.000000
13245211614,0.000000,-0.4,-0.571429,-0.142857,-0.232143,-0.304348,-0.285714,-0.1,-0.243902,-0.5,...,0.0,-0.178571,0.0,-0.125,-1.0,0.0,-0.4,-0.272727,-0.285714,-0.222222
13238590958,0.000000,0.4,0.000000,0.142857,0.017857,0.304348,0.000000,-0.1,0.000000,0.5,...,0.0,-0.035714,0.0,-0.125,1.0,0.0,0.4,0.090909,0.285714,0.222222
13237349458,0.000000,-0.4,-0.571429,-0.142857,-0.232143,-0.304348,-0.285714,-0.1,-0.243902,-0.5,...,0.0,-0.178571,0.0,-0.125,-1.0,0.0,-0.4,-0.272727,-0.285714,-0.222222


In [20]:
#define knn model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=3, n_jobs=-1)
model_knn.fit(train)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=3)

### Evaluation

In [21]:
test.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
13237751174,-0.333333,0.2,0.142857,0.000000,0.035714,0.086957,0.142857,0.0,0.000000,0.5,...,-1.0,-0.142857,0.0,0.25,0.0,0.0,0.2,-0.090909,0.285714,0.111111
13264663777,-0.666667,0.0,-0.142857,0.142857,0.035714,0.043478,-0.142857,-0.2,0.024390,0.0,...,0.0,0.035714,0.0,0.25,0.0,-1.0,-0.2,-0.181818,0.428571,0.111111
13249117061,-0.333333,0.2,-0.142857,0.000000,0.035714,0.173913,0.285714,0.0,-0.048780,-0.5,...,-1.0,-0.071429,0.0,0.00,0.0,0.0,-0.2,-0.272727,0.285714,-0.111111
13238048893,-0.666667,0.0,-0.142857,-0.714286,-0.214286,-0.304348,-0.142857,-0.2,-0.268293,0.0,...,0.0,-0.321429,0.0,-0.25,0.0,-1.0,-0.2,-0.181818,-0.428571,-0.333333
13239867945,-0.666667,0.0,-0.142857,-0.714286,-0.214286,-0.304348,-0.142857,-0.2,-0.268293,0.0,...,0.0,-0.321429,0.0,-0.25,0.0,-1.0,-0.2,-0.181818,-0.428571,-0.333333


In [22]:
#####voorbeeld hoe het werkt bij 1 persoon 
person1 = test.iloc[1,:].values.reshape(1,-1)

In [23]:
n_neighors = 3
n_recommendations = 5
# get predicted values
style_neighbors = model_knn.kneighbors(person1, n_neighors, return_distance=True)

In [24]:
#the 3 distances, followed up with 3 same users
style_neighbors

(array([[0.19914397, 0.22619298, 0.33500908]]), array([[47, 33, 29]]))

In [25]:
#the 3 neighbors
style_neighbors[1]

array([[47, 33, 29]])

In [26]:
style_neighbors[1][0]

array([47, 33, 29])

In [27]:
#de score van the neighbors voor 25 genres
train.iloc[style_neighbors[1][0]]

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
13263750995,-0.666667,0.0,-0.142857,-0.428571,-0.035714,-0.043478,-0.142857,-0.1,-0.024390,0.0,...,0.0,0.107143,0.0,0.00,0.0,-1.0,-0.2,0.000000,0.142857,0.111111
13237315738,-0.666667,0.0,-0.142857,0.142857,-0.142857,0.043478,-0.142857,-0.1,-0.170732,0.0,...,0.0,-0.107143,0.0,-0.25,0.0,-1.0,0.2,-0.181818,0.142857,0.111111
13239741363,0.000000,0.0,-0.142857,-0.142857,-0.071429,-0.043478,-0.142857,-0.2,-0.073171,0.0,...,0.0,-0.035714,0.0,-0.25,0.0,-1.0,0.2,-0.181818,0.142857,0.111111


In [28]:
#de mean score van de 3 soort gelijke users, voor de 25 style labels
avg_score_genres = train.iloc[style_neighbors[1][0]].mean()
avg_score_genres

0    -0.444444
1     0.000000
2    -0.142857
3    -0.142857
4    -0.083333
5    -0.014493
6    -0.142857
7    -0.133333
8    -0.089431
9     0.000000
10    0.000000
11    0.000000
12    0.058824
13   -0.148148
14   -0.333333
15    0.000000
16   -0.011905
17    0.000000
18   -0.166667
19    0.000000
20   -1.000000
21    0.066667
22   -0.121212
23    0.142857
24    0.111111
dtype: float64

In [29]:
def sort_index(lst, rev=True):
    index = range(len(lst))
    s = sorted(index, reverse=rev, key=lambda i: lst[i])
    return s

In [30]:
index_recommendations = sort_index(avg_score_genres)[:n_recommendations]
index_recommendations

[23, 24, 21, 12, 1]

In [31]:
style_recommendations = [style_indexes[rec] for rec in index_recommendations]
style_recommendations

['Romantic', 'Sexy', 'Party', 'Feminine', 'Bohemian']

####  Recommendations voor de hele test set 

In [32]:
style_neighbors = model_knn.kneighbors(test, n_neighors, return_distance=True)

In [33]:
#voor alle 25 personen in de test set de 3 neighbors
neighbors = style_neighbors[1]
neighbors

array([[52, 24, 28],
       [47, 33, 29],
       [44, 38,  9],
       [ 6, 13, 17],
       [ 6, 13, 17],
       [ 9, 12, 37],
       [ 6, 13, 17],
       [47, 57, 33],
       [15, 20, 50],
       [44, 38, 21],
       [40,  3,  1],
       [58,  7, 55],
       [40,  3,  1],
       [ 9, 12, 37],
       [27, 34, 10],
       [58,  7, 55],
       [46, 31, 30],
       [44, 38,  9],
       [30, 45, 31],
       [14, 26, 42],
       [15, 49,  8],
       [31, 30, 46],
       [21, 24, 19],
       [31, 30, 46],
       [ 6, 13, 17],
       [ 7, 58, 55]])

In [34]:
#voeg alle stappen zoals samen zoals bij 1 persoon: maar nu voor iedere persoon
#alle recommendations voor de test set
recommendations_test = [sort_index(train.iloc[user].mean())[:n_recommendations] for user in style_neighbors[1]]
recommendations_test

[[9, 18, 13, 2, 5],
 [23, 24, 21, 12, 1],
 [14, 1, 5, 3, 10],
 [1, 9, 10, 11, 15],
 [1, 9, 10, 11, 15],
 [3, 10, 11, 17, 19],
 [1, 9, 10, 11, 15],
 [24, 12, 21, 23, 1],
 [20, 14, 16, 2, 7],
 [14, 1, 5, 3, 6],
 [19, 9, 21, 1, 5],
 [17, 21, 18, 24, 5],
 [19, 9, 21, 1, 5],
 [3, 10, 11, 17, 19],
 [3, 5, 6, 22, 4],
 [17, 21, 18, 24, 5],
 [21, 13, 24, 12, 3],
 [14, 1, 5, 3, 10],
 [3, 21, 12, 13, 24],
 [0, 9, 11, 14, 15],
 [20, 2, 21, 1, 9],
 [21, 13, 24, 12, 3],
 [9, 13, 18, 8, 3],
 [21, 13, 24, 12, 3],
 [1, 9, 10, 11, 15],
 [17, 21, 18, 24, 5]]

In [35]:
i = 0
for recommendation in recommendations_test:
    print("User: " + test.index[i] + " heeft aanbevelingen:")
    print([style_indexes[rec] for rec in recommendation])
    i += 1


User: 13237751174 heeft aanbevelingen:
['Contemporary', 'Minimalistic', 'Form-fitting', 'Bold', 'Chic']
User: 13264663777 heeft aanbevelingen:
['Romantic', 'Sexy', 'Party', 'Feminine', 'Bohemian']
User: 13249117061 heeft aanbevelingen:
['Formal', 'Bohemian', 'Chic', 'Business', 'Designer']
User: 13238048893 heeft aanbevelingen:
['Bohemian', 'Contemporary', 'Designer', 'Eclectic', 'Goth']
User: 13239867945 heeft aanbevelingen:
['Bohemian', 'Contemporary', 'Designer', 'Eclectic', 'Goth']
User: 13237445953 heeft aanbevelingen:
['Business', 'Designer', 'Eclectic', 'Luxury', 'Modest']
User: 13240301382 heeft aanbevelingen:
['Bohemian', 'Contemporary', 'Designer', 'Eclectic', 'Goth']
User: 13238520630 heeft aanbevelingen:
['Sexy', 'Feminine', 'Party', 'Romantic', 'Bohemian']
User: 13244840492 heeft aanbevelingen:
['Monochrome', 'Formal', 'Loose-fit', 'Bold', 'Colorful']
User: 13263660860 heeft aanbevelingen:
['Formal', 'Bohemian', 'Chic', 'Business', 'Classic']
User: 13237449180 heeft aanbev

In [36]:
#get 3 actual most popular genres
y_test = test.apply(lambda x: pd.Series(x.nlargest(5).index), axis=1)
y_test = y_test.values.tolist()

In [37]:
#flatten (nested) list to evaluate
y_pred = [val for sublist in recommendations_test for val in sublist]
y_test = [val for sublist in y_test for val in sublist]

In [38]:
metrics.f1_score(y_test,y_pred,average = 'macro')

0.3986474519632414